In [1]:
import pandas as pd

In [2]:
categories = pd.read_excel('Object Keyword grouping.xlsx',sheet_name = 'Organized - Final')

#### Others

In [4]:
others_category = categories.iloc[:,0]

In [5]:
others_mapping = pd.DataFrame({'Mapping':'Others' ,'Object':others_category[~others_category.isna()]})

#### Main Categories

In [7]:
all_categories = categories.iloc[:,4:].melt()

In [8]:
all_categories = all_categories[~all_categories.value.isna()]

to_be_removed = all_categories.variable[all_categories.variable.str.startswith('Count')].unique()

all_categories_map = all_categories[~all_categories.variable.isin(to_be_removed)]
all_categories_map.reset_index(inplace = True, drop = True)

all_categories_map.columns = ['Mapping','Object']

In [9]:
mapping_file = pd.concat([others_mapping,all_categories_map],axis = 0)

#### Reading the actual objects in chinese and english

In [37]:
with open('Translation_all.txt','r',encoding ='utf-8') as f:
    translation =f.readlines()
    
translation = [each.strip() for each in translation]

In [103]:
with open('words_all.txt','r',encoding ='utf-8') as f:
    chinese =f.readlines()
chinese = [each.strip() for each in chinese]

In [104]:
chinese_eng = pd.DataFrame({'Chinese':chinese,'Translation':translation})

In [105]:
chinese_eng.shape

(4492, 2)

In [106]:
mapping_file['Obj_clean'] = mapping_file.Object.str.lower().str.replace(' ','')
chinese_eng['Translation_clean'] = chinese_eng.Translation.str.lower().str.replace(' ','')

In [107]:
object_map_merge = pd.merge(chinese_eng,mapping_file,how = 'left',left_on = 'Translation_clean',right_on='Obj_clean')

In [108]:
object_map_merge.Mapping = object_map_merge.Mapping.fillna('Others').str.replace(' ','_')

In [109]:
object_final_map = object_map_merge.loc[:,['Chinese','Mapping']]

In [111]:
mapping_dict = pd.Series(object_final_map.Mapping.values,object_final_map.Chinese).to_dict()

In [122]:
chinese_eng.head()

,Chinese,Translation,Translation_clean
0,0,0,0
1,"""""","""""",""""""
2,老虎头,Tiger head,tigerhead
3,背景词,Background word,backgroundword
4,白菜,Chinese cabbage,chinesecabbage


In [126]:
mapping_dict_english_chinese =  pd.Series(chinese_eng.Translation.values,chinese_eng.Chinese.values).to_dict()

#### Reading the response file

In [132]:
alldata = pd.read_excel("AllData.xlsx")

In [135]:
alldata['Obj_split'] = alldata['Objects'].str.split("，")

In [136]:
alldata.Obj_split = alldata.Obj_split.fillna('')

In [137]:
mapping_dict['人鸟'] = 'Human bird'
mapping_dict['水母人头'] = 'Jellyfish head'

#### Converting all the chinese object names to english

In [138]:
import string
punct = string.punctuation
punct = punct+'（？）'

In [139]:
respone_obj_english = alldata.Obj_split.apply(lambda x:
                             [mapping_dict_english_chinese[''.join([letter for letter in list(each) if 
                                                    not letter.isdigit() and letter != 'M' 
                                                    and  letter != ' ' and letter not in punct])] if
                              
                              ''.join([letter for letter in list(each) if 
                                                    not letter.isdigit() and letter != 'M' 
                                                    and  letter != ' ' and letter not in punct]) in mapping_dict_english_chinese.keys() else 
                                       'Others' 
                              
                              for each in x])

In [140]:
respone_obj_english.shape

(30003,)

In [146]:
alldata['Objects_english'] = respone_obj_english
alldata_output = alldata.copy()

In [148]:
alldata_output = alldata_output.drop(['Obj_split'],axis = 1)

In [151]:
alldata_output.to_excel('AllData_clean.xlsx',index = False)

### Grouping objects

In [74]:
respone_obj = alldata.Obj_split.apply(lambda x:
                             [mapping_dict[''.join([letter for letter in list(each) if 
                                                    not letter.isdigit() and letter != 'M' 
                                                    and  letter != ' ' and letter not in punct])] if
                              
                              ''.join([letter for letter in list(each) if 
                                                    not letter.isdigit() and letter != 'M' 
                                                    and  letter != ' ' and letter not in punct]) in mapping_dict.keys() else 
                                       'Others' 
                              
                              for each in x])

In [79]:
respone_obj.apply(lambda x: len([each for each in x if each == 'Others'])).sum()

1393

In [80]:
respone_obj.apply(lambda x: len([each for each in x if each != 'Others'])).sum()

15691

In [93]:
object_response = respone_obj.apply(lambda x: 'XXANDXX'.join(x)).str.replace(",","_").str.replace("__","_").str.replace("XXANDXX",", ")

In [126]:
object_response[object_response == ''] = 'Others'

In [127]:
alldata['Object_respone'] = object_response

In [128]:
alldata_final = alldata.loc[:,['图片编号','Object_respone']]

In [129]:
alldata_final.columns = ['Image_index','Object_respone']

In [130]:
#alldata_final.sort_values('Image_index')

In [131]:
alldata_final.to_csv('Object_Multilable.csv',index = False)